This file uses one of the datasets (`Niveau_de_vie_2013_a_la_commune.xlsx`) given as an input and tries to fit a simple linear regression (which may not be very appropriate here).

# Exploration des Données

In [1]:
import pandas as pd

df = pd.read_csv("datasets/results_train.csv", dtype={"CodeINSEE": str})
df


,CodeINSEE,Libellé du département,Libellé de la commune,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,...,Voix,% Voix/Ins,% Voix/Exp,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,01006,Ain,Ambléon,Complet,103,19,18.45,84,81.55,12,...,45,43.69,62.50,2,F,LE PEN,Marine,27,26.21,37.50
1,01009,Ain,Andert-et-Condon,Complet,280,73,26.07,207,73.93,22,...,102,36.43,56.04,2,F,LE PEN,Marine,80,28.57,43.96
2,01010,Ain,Anglefort,Complet,792,185,23.36,607,76.64,31,...,227,28.66,39.96,2,F,LE PEN,Marine,341,43.06,60.04
3,01011,Ain,Apremont,Complet,289,42,14.53,247,85.47,29,...,101,34.95,47.20,2,F,LE PEN,Marine,113,39.10,52.80
4,01012,Ain,Aranc,Complet,283,76,26.86,207,73.14,28,...,92,32.51,53.18,2,F,LE PEN,Marine,81,28.62,46.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20887,95652,Val-d'Oise,Viarmes,Complet,4026,1106,27.47,2920,72.53,165,...,1422,35.32,52.71,2,F,LE PEN,Marine,1276,31.69,47.29
20888,95656,Val-d'Oise,Vienne-en-Arthies,Complet,332,58,17.47,274,82.53,22,...,142,42.77,57.49,2,F,LE PEN,Marine,105,31.63,42.51
20889,95680,Val-d'Oise,Villiers-le-Bel,Complet,12414,5099,41.07,7315,58.93,452,...,4541,36.58,67.76,2,F,LE PEN,Marine,2161,17.41,32.24
20890,95682,Val-d'Oise,Villiers-le-Sec,Complet,110,26,23.64,84,76.36,3,...,50,45.45,62.50,2,F,LE PEN,Marine,30,27.27,37.50


In [7]:
# Load the additional dataset
niveau_vie = pd.read_excel('datasets/Niveau_de_vie_2013_a_la_commune.xlsx')
communes = pd.read_csv('datasets/communes-france-2022.csv')
ages = pd.read_excel('datasets/age-insee-2020.xlsx')
mdb = pd.read_excel('datasets/MDB-INSEE-V2.xls')


C:\Users\lance\AppData\Local\Temp\ipykernel_23796\162448712.py:3: DtypeWarning: Columns (1,12,14,16,23) have mixed types. Specify dtype option on import or set low_memory=False.
  communes = pd.read_csv('datasets/communes-france-2022.csv')


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [ ]:
niveau_vie

,Code Commune,Nom Commune,Niveau de vie Commune,Niveau de vie Département
0,05047,Éourres,10021.250000,19202.516480
1,26142,Glandage,10215.000000,19300.504181
2,11317,Rodome,10908.500000,17599.733068
3,11384,Soulatgé,11485.166667,17599.733068
4,30153,Malons-et-Elze,11680.000000,18282.543539
...,...,...,...,...
36567,91526,Roinvilliers,NaN,23141.927413
36568,74203,Novel,NaN,24707.911366
36569,78264,Gambaiseuil,NaN,26305.343074
36570,78606,Le Tartre-Gaudran,NaN,26305.343074


In [ ]:


# Replace the Code Commune by CodeINSEE name and ensure it is a string on 5 characters
niveau_vie['Code Commune'] = niveau_vie['Code Commune'].apply(lambda x: str(x).zfill(5))
niveau_vie = niveau_vie.rename(columns={'Code Commune': 'CodeINSEE'})
niveau_vie['CodeINSEE'] = niveau_vie['CodeINSEE'].astype(str)
assert all(niveau_vie['CodeINSEE'].apply(lambda x: len(x) == 5))
assert all(df['CodeINSEE'].apply(lambda x: len(x) == 5))
niveau_vie

In [ ]:
# Join the two dataframes on the INSEE code.
df = df.merge(niveau_vie, on='CodeINSEE', how='left')
df

,CodeINSEE,Libellé du département,Libellé de la commune,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Nom Commune,Niveau de vie Commune,Niveau de vie Département
0,01006,Ain,Ambléon,Complet,103,19,18.45,84,81.55,12,...,2,F,LE PEN,Marine,27,26.21,37.50,Ambléon,21871.666667,22343.574665
1,01009,Ain,Andert-et-Condon,Complet,280,73,26.07,207,73.93,22,...,2,F,LE PEN,Marine,80,28.57,43.96,Andert-et-Condon,21486.250000,22343.574665
2,01010,Ain,Anglefort,Complet,792,185,23.36,607,76.64,31,...,2,F,LE PEN,Marine,341,43.06,60.04,Anglefort,21015.333333,22343.574665
3,01011,Ain,Apremont,Complet,289,42,14.53,247,85.47,29,...,2,F,LE PEN,Marine,113,39.10,52.80,Apremont,22690.000000,22343.574665
4,01012,Ain,Aranc,Complet,283,76,26.86,207,73.14,28,...,2,F,LE PEN,Marine,81,28.62,46.82,Aranc,20211.000000,22343.574665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20887,95652,Val-d'Oise,Viarmes,Complet,4026,1106,27.47,2920,72.53,165,...,2,F,LE PEN,Marine,1276,31.69,47.29,Viarmes,24955.428571,21296.219805
20888,95656,Val-d'Oise,Vienne-en-Arthies,Complet,332,58,17.47,274,82.53,22,...,2,F,LE PEN,Marine,105,31.63,42.51,Vienne-en-Arthies,25266.000000,21296.219805
20889,95680,Val-d'Oise,Villiers-le-Bel,Complet,12414,5099,41.07,7315,58.93,452,...,2,F,LE PEN,Marine,2161,17.41,32.24,Villiers-le-Bel,14212.857143,21296.219805
20890,95682,Val-d'Oise,Villiers-le-Sec,Complet,110,26,23.64,84,76.36,3,...,2,F,LE PEN,Marine,30,27.27,37.50,Villiers-le-Sec,25350.476190,21296.219805


In [ ]:
# There are some missing values in the additional dataset
df.isnull().sum()

CodeINSEE                       0
Libellé du département          0
Libellé de la commune           0
Etat saisie                     0
Inscrits                        0
Abstentions                     0
% Abs/Ins                       0
Votants                         0
% Vot/Ins                       0
Blancs                          0
% Blancs/Ins                    0
% Blancs/Vot                    0
Nuls                            0
% Nuls/Ins                      0
% Nuls/Vot                      0
Exprimés                        0
% Exp/Ins                       0
% Exp/Vot                       0
N°Panneau                       0
Sexe                            0
Nom                             0
Prénom                          0
Voix                            0
% Voix/Ins                      0
% Voix/Exp                      0
Unnamed: 26                     0
Unnamed: 27                     0
Unnamed: 28                     0
Unnamed: 29                     0
Unnamed: 30   

In [ ]:
# Define the model pipeline.
# Fill missing values, normalize the predictor columns, and then apply Linear Regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

predictor_columns = ['Inscrits', 'Niveau de vie Commune', 'Niveau de vie Département']

X = df[predictor_columns]
y = df['% Voix/Ins']

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), # Replace missing values with the mean
    ('scaler', StandardScaler()), # Normalize the data
    ('model', LinearRegression())
])

# Train the pipeline (without leaking any information from the test set)
pipeline.fit(X, y)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('model', LinearRegression())])

In [ ]:
# Load the test set and apply the same transformations (careful not to fit the pipeline on the test set!)
df_test = pd.read_csv('datasets/results_test.csv', dtype={"CodeINSEE": str})
df_test = df_test.merge(niveau_vie, on='CodeINSEE', how='left')

# Predict the results
X_test = df_test[predictor_columns]
y_pred = pipeline.predict(X_test)
df_test['Prediction'] = y_pred

In [ ]:
# Prepare the Kaggle submission
df_test = df_test[['CodeINSEE', 'Prediction']]
df_test.to_csv('results_test_predicted.csv', index=False)
df_test

,CodeINSEE,Prediction
0,01001,38.173184
1,01002,39.291330
2,01004,36.226083
3,01005,38.513582
4,01007,37.798632
...,...,...
13923,95658,42.888149
13924,95660,42.620477
13925,95675,42.353480
13926,95676,41.739699
